In [1]:
!pip install numpy pandas gurobipy

In [1]:
# === Setup ===
import numpy as np
import pandas as pd
from gurobipy import Model, GRB, quicksum

# Fix random seed
np.random.seed(42)

# === Data ===

# Specialist time distributions
specialists = {
    'S1': (7.3, 1.0), 'S2': (4.0, 0.6), 'S3': (6.0, 1.2), 'S4': (7.5, 1.5),
    'S5': (5.0, 0.8), 'S6': (6.8, 1.3), 'S7': (3.5, 1.0), 'S8': (7.0, 1.1),
    'S9': (6.5, 1.0), 'S10': (5.5, 1.4)
}

# Room cost data
rooms = {
    'O1': {'regular_cost': 33, 'overtime_cost': 50},
    'O2': {'regular_cost': 28, 'overtime_cost': 45},
    'O3': {'regular_cost': 35, 'overtime_cost': 52},
    'O4': {'regular_cost': 30, 'overtime_cost': 47}
}

REGULAR_TIME = 12
NUM_DAYS = 500
room_names = list(rooms.keys())
specialist_names = list(specialists.keys())

# === Simulate 500 Days of Specialist Time ===
def simulate_days():
    data = {}
    for s, (mean, std) in specialists.items():
        data[s] = np.maximum(np.random.normal(loc=mean, scale=std, size=NUM_DAYS), 0)
    return pd.DataFrame(data)

sim_data = simulate_days()

# === MILP Solver for One Day ===
def solve_day(specialist_times, max_overtime):
    model = Model("OR_Scheduling")
    model.setParam('OutputFlag', 0)  # silence output

    x = model.addVars(specialist_names, room_names, vtype=GRB.BINARY, name="x")
    overtime = model.addVars(room_names, vtype=GRB.CONTINUOUS, lb=0, name="overtime")

    # Objective
    model.setObjective(
        quicksum(rooms[r]['regular_cost'] for r in room_names) +
        quicksum(overtime[r] * rooms[r]['overtime_cost'] for r in room_names),
        GRB.MINIMIZE
    )

    # Constraints
    for s in specialist_names:
        model.addConstr(quicksum(x[s, r] for r in room_names) == 1)

    for r in room_names:
        model.addConstr(quicksum(x[s, r] for s in specialist_names) <= 3)

    # Equipment conflicts
    for r in room_names:
        model.addConstr(x['S2', r] + x['S7', r] <= 1)
        model.addConstr(x['S4', r] + x['S5', r] <= 1)

    for r in room_names:
        total_time = quicksum(specialist_times[s] * x[s, r] for s in specialist_names)
        model.addConstr(total_time <= REGULAR_TIME + overtime[r])
        model.addConstr(overtime[r] <= max_overtime)

    model.optimize()

    if model.status == GRB.OPTIMAL:
        total_cost = model.objVal
        overtime_usage = {r: overtime[r].X for r in room_names}
        return total_cost, overtime_usage
    else:
        return None, None

# === Run Simulation for Each Policy ===
def evaluate_policy(max_overtime):
    daily_costs = []
    daily_overtimes = {r: [] for r in room_names}

    for day in range(NUM_DAYS):
        specialist_times = sim_data.iloc[day].to_dict()
        cost, ot = solve_day(specialist_times, max_overtime)
        if cost is not None:
            daily_costs.append(cost)
            for r in room_names:
                daily_overtimes[r].append(ot[r])
        else:
            print(f"Day {day} failed to solve.")

    return daily_costs, pd.DataFrame(daily_overtimes)

# === Run all three policies ===
policies = {
    "Policy 1 (3h)": 3,
    "Policy 2 (5h)": 5,
    "Policy 3 (7h)": 7
}

results = {}

for name, limit in policies.items():
    print(f"Running {name}...")
    costs, overtimes = evaluate_policy(limit)
    results[name] = {
        "costs": costs,
        "overtimes": overtimes,
        "mean_cost": np.mean(costs),
        "max_ot": overtimes.max(),
        "mean_ot": overtimes.mean()
    }

# === Show Results Summary ===
for name in policies:
    print(f"\n{name}:")
    print(f"  Average Cost: {results[name]['mean_cost']:.2f}")
    print(f"  Max Overtime Per Room:\n{results[name]['max_ot']}")
    print(f"  Mean Overtime Per Room:\n{results[name]['mean_ot']}")


Running Policy 1 (3h)...
Set parameter Username
Set parameter LicenseID to value 2634663
Academic license - for non-commercial use only - expires 2026-03-10
Day 0 failed to solve.
Day 1 failed to solve.
Day 3 failed to solve.
Day 6 failed to solve.
Day 7 failed to solve.
Day 8 failed to solve.
Day 11 failed to solve.
Day 12 failed to solve.
Day 15 failed to solve.
Day 18 failed to solve.
Day 20 failed to solve.
Day 21 failed to solve.
Day 23 failed to solve.
Day 25 failed to solve.
Day 26 failed to solve.
Day 27 failed to solve.
Day 28 failed to solve.
Day 30 failed to solve.
Day 32 failed to solve.
Day 34 failed to solve.
Day 35 failed to solve.
Day 36 failed to solve.
Day 40 failed to solve.
Day 41 failed to solve.
Day 43 failed to solve.
Day 44 failed to solve.
Day 45 failed to solve.
Day 46 failed to solve.
Day 47 failed to solve.
Day 48 failed to solve.
Day 49 failed to solve.
Day 52 failed to solve.
Day 53 failed to solve.
Day 55 failed to solve.
Day 57 failed to solve.
Day 58 fa

In [2]:
def solve_day_with_scenario(specialist_times, max_overtime, scenario=None, fixed_room=None):
    model = Model("OR_Scheduling_Scenario")
    model.setParam('OutputFlag', 0)

    x = model.addVars(specialist_names, room_names, vtype=GRB.BINARY, name="x")
    overtime = model.addVars(room_names, vtype=GRB.CONTINUOUS, lb=0, name="overtime")

    # Objective function
    model.setObjective(
        quicksum(rooms[r]['regular_cost'] for r in room_names) +
        quicksum(overtime[r] * rooms[r]['overtime_cost'] for r in room_names),
        GRB.MINIMIZE
    )

    # Basic constraints
    for s in specialist_names:
        model.addConstr(quicksum(x[s, r] for r in room_names) == 1)

    for r in room_names:
        model.addConstr(quicksum(x[s, r] for s in specialist_names) <= 3)

    # Equipment conflicts
    for r in room_names:
        model.addConstr(x['S2', r] + x['S7', r] <= 1)
        model.addConstr(x['S4', r] + x['S5', r] <= 1)

    for r in room_names:
        total_time = quicksum(specialist_times[s] * x[s, r] for s in specialist_names)
        model.addConstr(total_time <= REGULAR_TIME + overtime[r])
        model.addConstr(overtime[r] <= max_overtime)

    # Scenario-specific constraints
    if scenario == 1:
        model.addConstr(overtime['O3'] == 0)
    elif scenario == 2:
        model.addConstr(x['S4', 'O1'] == 0)
        model.addConstr(x['S4', 'O2'] == 0)
    elif scenario == 3:
        if fixed_room is not None:
            for r in room_names:
                if r != fixed_room:
                    model.addConstr(x['S4', r] == 0)

    model.optimize()

    if model.status == GRB.OPTIMAL:
        total_cost = model.objVal
        overtime_usage = {r: overtime[r].X for r in room_names}
        s4_room = [r for r in room_names if x['S4', r].X > 0.5][0]
        return total_cost, overtime_usage, s4_room
    else:
        return None, None, None


In [3]:
def evaluate_scenario(policy_limit, scenario=None, fixed_room=None):
    daily_costs = []
    daily_overtimes = {r: [] for r in room_names}
    s4_rooms = []

    for day in range(NUM_DAYS):
        specialist_times = sim_data.iloc[day].to_dict()
        cost, ot, s4_room = solve_day_with_scenario(specialist_times, policy_limit, scenario, fixed_room)
        if cost is not None:
            daily_costs.append(cost)
            for r in room_names:
                daily_overtimes[r].append(ot[r])
            s4_rooms.append(s4_room)
        else:
            print(f"Day {day} failed.")

    return daily_costs, pd.DataFrame(daily_overtimes), s4_rooms


In [5]:
best_policy = min(results.items(), key=lambda x: x[1]["mean_cost"])
print(f"\n✅ Best policy is: {best_policy[0]} with average cost = {best_policy[1]['mean_cost']:.2f}")
BEST_POLICY_LIMIT = policies[best_policy[0]]  # Use for Part B



✅ Best policy is: Policy 1 (3h) with average cost = 501.68


In [7]:
# Policy 1 (3h) was the best. 
BEST_POLICY_LIMIT = 3

# Scenario 1: No overtime for O3
costs1, ot1, _ = evaluate_scenario(BEST_POLICY_LIMIT, scenario=1)
print(f"Scenario 1 - Avg Cost: {np.mean(costs1):.2f}")

# Scenario 2: S4 can only be in OR3 or OR4
costs2, ot2, _ = evaluate_scenario(BEST_POLICY_LIMIT, scenario=2)
print(f"Scenario 2 - Avg Cost: {np.mean(costs2):.2f}")

# Scenario 3: S4 must stay in same room daily
_, _, s4_base_rooms = evaluate_scenario(BEST_POLICY_LIMIT)  # get baseline S4 room
from collections import Counter
most_common_room = Counter(s4_base_rooms).most_common(1)[0][0]

costs3, ot3, _ = evaluate_scenario(BEST_POLICY_LIMIT, scenario=3, fixed_room=most_common_room)
print(f"Scenario 3 - Avg Cost: {np.mean(costs3):.2f}")

Day 0 failed.
Day 1 failed.
Day 2 failed.
Day 3 failed.
Day 6 failed.
Day 7 failed.
Day 8 failed.
Day 9 failed.
Day 10 failed.
Day 11 failed.
Day 12 failed.
Day 14 failed.
Day 15 failed.
Day 16 failed.
Day 18 failed.
Day 20 failed.
Day 21 failed.
Day 22 failed.
Day 23 failed.
Day 24 failed.
Day 25 failed.
Day 26 failed.
Day 27 failed.
Day 28 failed.
Day 30 failed.
Day 32 failed.
Day 34 failed.
Day 35 failed.
Day 36 failed.
Day 39 failed.
Day 40 failed.
Day 41 failed.
Day 42 failed.
Day 43 failed.
Day 44 failed.
Day 45 failed.
Day 46 failed.
Day 47 failed.
Day 48 failed.
Day 49 failed.
Day 50 failed.
Day 52 failed.
Day 53 failed.
Day 54 failed.
Day 55 failed.
Day 57 failed.
Day 58 failed.
Day 59 failed.
Day 60 failed.
Day 61 failed.
Day 63 failed.
Day 64 failed.
Day 65 failed.
Day 67 failed.
Day 68 failed.
Day 69 failed.
Day 70 failed.
Day 71 failed.
Day 72 failed.
Day 73 failed.
Day 74 failed.
Day 75 failed.
Day 76 failed.
Day 77 failed.
Day 78 failed.
Day 79 failed.
Day 80 failed.
Day